# Image Preprocessing

In [1]:
#importing keras library
import keras

In [2]:
#importing the image data generator
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [3]:
#Defining the parameter for image generator class
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,rotation_range=180,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [4]:
#Applying image data generator functionality to train set 
x_train=train_datagen.flow_from_directory(r'D:\IBM\Dataset\train_set',
target_size=(128,128),batch_size=32,class_mode='binary')

Found 436 images belonging to 2 classes.


In [5]:
#Applying image data generator functionality to test set
x_test=test_datagen.flow_from_directory(r'D:\IBM\Dataset\test_set',
target_size=(128, 128),batch_size=32,class_mode='binary')


Found 121 images belonging to 2 classes.


# Model Building

In [6]:
#To define linear intialisation import Sequential 
from keras.models import Sequential
#To add layers import Dense
from keras.layers import Dense
#To creat Convolution kernal import Convolution2D
from keras.layers import Convolution2D
#import Maxpooling layer
from keras.layers import MaxPooling2D
#import Flatten layer
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

In [7]:
#initializing the model
model=Sequential()
#add convolution layer
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#add convolution layer
model.add(Convolution2D(64,(3,3),activation='relu'))
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#add convolution layer
model.add(Convolution2D(128,(3,3),activation='relu'))
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#add convolution layer
model.add(Convolution2D(128,(3,3),activation='relu'))
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#add flatten layer
model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [8]:
#configuring the learning process
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])

In [9]:
#Training the model
r=model.fit(x_train,epochs=10,validation_data=x_test)

Epoch 1/10
14/14 [==============================] - 47s 3s/step - loss: 0.7029 - accuracy: 0.5757 - val_loss: 0.6005 - val_accuracy: 0.5950
Epoch 2/10
14/14 [==============================] - 46s 3s/step - loss: 0.5108 - accuracy: 0.7982 - val_loss: 0.2414 - val_accuracy: 0.9256
Epoch 3/10
14/14 [==============================] - 47s 3s/step - loss: 0.2793 - accuracy: 0.8784 - val_loss: 0.2167 - val_accuracy: 0.9091
Epoch 4/10
14/14 [==============================] - 48s 4s/step - loss: 0.2330 - accuracy: 0.8991 - val_loss: 0.0956 - val_accuracy: 0.9421
Epoch 5/10
14/14 [==============================] - 56s 4s/step - loss: 0.1715 - accuracy: 0.9197 - val_loss: 0.0257 - val_accuracy: 0.9917
Epoch 6/10
14/14 [==============================] - 58s 4s/step - loss: 0.1829 - accuracy: 0.9174 - val_loss: 0.0597 - val_accuracy: 1.0000
Epoch 7/10
14/14 [==============================] - 59s 4s/step - loss: 0.1714 - accuracy: 0.9289 - val_loss: 0.0249 - val_accuracy: 1.0000
Epoch 8/10
14/14 [==

In [30]:
#save the model
model.save("forestalert.h5")

# Video Analysis

In [31]:
from twilio.rest import Client

In [32]:
#import load model from keras.model
from keras.models import load_model
#import image from keras
from tensorflow.keras.preprocessing import image
import numpy as np
#import cv2
import cv2
#load the saved model
model=load_model(r"forestalert.h5")
img=image.load_img(r'D:\IBM\Dataset\test_set\with fire\Uttarakhand_forest_fire.jpeg')
x=image.img_to_array(img)
# res=cv2.resize(x,dsize=(150,150),interpolation=cv2.INTER_CUBIC)
#expand the image shape
x=np.expand_dims(x,axis=0)

In [33]:
from logging import WARNING
#import opencv library
import cv2
#import numpy
import numpy as np
#import image function from keras
from keras.preprocessing import image
#import load_model from keras
from keras.models import load_model
#import client from twilio API
from twilio.rest import Client
#import playsound package
from playsound import playsound

In [34]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
import tensorflow
from tensorflow.keras.preprocessing import image
from keras.models import load_model

In [ ]:
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
video = cv2.VideoCapture(r'C:\Users\Dell\Downloads\forest trees.mp4')
name=['forest','with fire']

In [ ]:
while(1):
 success,frame = video.read()
 cv2.imwrite("image.jpg",frame)
 img = tensorflow.keras.utils.load_img("image.jpg",target_size = (128,128))
 x = image.img_to_array(img)
 x = np.expand_dims(x,axis = 0)
 pred = model.predict(x)
 pred = pred[0][0]
 if pred > 0.5:
   pred = 1
 else :
    pred = 0
 print(pred)
 cv2.putText(frame,"predicted class = "+str(name[pred]),(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
 if pred==1:
  account_sid = 'ACab5b7ac22466b88a9cda7cf5414b750a'
  auth_token = 'c9c95130eade17e5e3d3f936283bef7a'
  client = Client(account_sid, auth_token)
  message = client.messages \
    .create(
    body='Danger!Forest Fire is detected!',
    from_='+17088477470',
    to='+918825826199')
  print(message.sid)
  print("Fire detected")
  print("SMS Sent!") 
  from playsound import playsound
  playsound(r"C:\Users\Dell\Downloads\alert alarm.wav")
  cv2.imshow('image',frame)
  if cv2.waitKey(0)&0xFF == ord('a'):
        break
 else:
   print("No Danger")
   cv2.imshow('image',frame)
   if cv2.waitKey(0)&0xFF == ord('a'):   
        break
 video.release()       
 cv2.destroyAllWindows() 

In [ ]:
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
video = cv2.VideoCapture(r"C:\Users\Dell\Downloads\Wild fire.mp4")
name=['forest','with fire']

In [53]:
while(1):
 success,frame = video.read()
 cv2.imwrite("image.jpg",frame)
 img = tensorflow.keras.utils.load_img("image.jpg",target_size = (128,128))
 x = image.img_to_array(img)
 x = np.expand_dims(x,axis = 0)
 pred = model.predict(x)
 pred = pred[0][0]
 if pred > 0.5:
   pred = 1
 else :
    pred = 0
 print(pred)
 cv2.putText(frame,"predicted class = "+str(name[pred]),(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
 if pred==1:
  account_sid = 'ACab5b7ac22466b88a9cda7cf5414b750a'
  auth_token = 'c9c95130eade17e5e3d3f936283bef7a'
  client = Client(account_sid, auth_token)
  message = client.messages \
    .create(
    body='Forest Fire is detected,Stay alert',
    from_='+17088477470',
    to='+918825826199')
  print(message.sid)
  print("Fire detected")
  print("SMS Sent!") 
  from playsound import playsound
  playsound(r"C:\Users\Dell\Downloads\alarm.mp3")
  cv2.imshow('image',frame)
  if cv2.waitKey(0)&0xFF == ord('a'):
        break
 else:
   print("No Danger")
   cv2.imshow('image',frame)
   if cv2.waitKey(0)&0xFF == ord('a'):   
        break
 video.release()       
 cv2.destroyAllWindows() 

1/1 [==============================] - 0s 48ms/step
1
SM7f8c0d730b291efe4bad93f913e44ecf
Fire detected
SMS Sent!


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'
